In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate
from dsgtools import azure

In [2]:
path = "Analytics/Customer Projects/RND_multiple_clint_rv6_attr_rerun - proj 12092 (RV 6.0)/Processing/ln_12092_cons1_input_riskview_v6_normal_total.csv"
data = pd.read_adls(path, reader = pd.read_csv, encoding='iso-8859-1', verbose = False, low_memory = False,)
print(data.shape)
bpl = pd.read_adls("Analytics/Customer Projects/Big Picture Loans - proj 11658 (FI, RV 6)/Processing/20231127/bpl_11658_rv60_attribs_retro.csv.gz", compression = "gzip",
                   reader = pd.read_csv, encoding = "iso-8859-1", verbose = False, low_memory = False,)
print(bpl.shape)
bpl["acctno"] = bpl.index
print([x for x in data.columns if x not in bpl.columns])
print([x for x in bpl.columns if x not in data.columns])
data = pd.concat([data, bpl], ignore_index=True)
print(data.shape)

(179684, 507)
(32011, 535)
['p_inpaddrcityflag']
['Unnamed: 0', 'transaction_id', 'company_id', 'date_added', 'dl_mask', 'exclude_dmv_pii', 'scout_opt_out', 'archive_opt_in', 'response_time', 'data_restriction_mask', 'data_permission_mask', 'i_ssn', 'i_name_first', 'i_name_last', 'i_address', 'i_city', 'i_state', 'i_zip', 'glb', 'dppa', 'i_dob', 'i_home_phone', 'year', 'month', 'day', 'IPAddress', 'Email', 'p_inpaddrcityflag_x', 'p_inpaddrcityflag_y']
(211695, 536)


In [3]:
prod_path = "Analytics/Customer Projects/RND_multiple_clint_rv6_attr_rerun - proj 12092 (RV 6.0)/File from the Customer/prod_oct_2023.parquet"
data_prod = pd.read_adls(prod_path)
print(data_prod.shape)
bpl = pd.read_adls("Analytics/Customer Projects/ACM Jira Tickets/ancuma_821_bpl__rv6attr/df_prod.parquet")
print(bpl.shape)
bpl.transaction_id = bpl.index
print([x for x in data_prod.columns if x not in bpl.columns])
print([x for x in bpl.columns if x not in data_prod.columns])
data_prod = pd.concat([data_prod, bpl], ignore_index=True)
print(data_prod.shape)

(179684, 402)
(32011, 403)
['industry', 'i_dl', 'ymd']
['date_format(date_added, yyyy-MM-dd)', 'date_added_ymd', 'date_added_ym', 'date_added_y']
(211695, 406)


In [124]:
freq(data_prod.date_added.astype(str).str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
date_added,,,,
2023-08,3891.0,0.018631,3891.0,0.018631
2023-09,11109.0,0.053194,15000.0,0.071825
2023-10,188482.0,0.902514,203482.0,0.974339
2023-11,5359.0,0.025661,208841.0,1.000000


In [30]:
data_prod.columns[:50]

Index(['transaction_id', 'company_id', 'login_id', 'product_id',
       'function_name', 'esp_method', 'delivery_method', 'date_added',
       'dl_mask', 'exclude_dmv_pii', 'scout_opt_out', 'archive_opt_in',
       'response_time', 'data_restriction_mask', 'data_permission_mask',
       'industry', 'i_ssn', 'i_name_first', 'i_name_last', 'i_lexid',
       'i_address', 'i_city', 'i_state', 'i_zip', 'i_dl', 'i_attributes_name',
       'o_lexid', 'glb', 'dppa', 'i_dob', 'i_home_phone', 'year', 'month',
       'day', 'tbl_refreshed_date_UTC', 'ymd', 'Email', 'IPAddress',
       'p_inpaddrcityflag', 'p_inpaddrstateflag', 'p_inpaddrstflag',
       'p_inpaddrzipflag', 'p_inpdlflag', 'p_inpdlstateflag', 'p_inpdobflag',
       'p_inplexidflag', 'p_inpnamefirstflag', 'p_inpnamelastflag',
       'p_inpphonehomeflag', 'p_inpssnflag'],
      dtype='object')

In [6]:
freq(data_prod.company_id)

,Count,Pct,Cuml Count,Cuml Pct
company_id,,,,
107571,43516.0,0.205560,43516.0,0.205560
107671,27572.0,0.130244,71088.0,0.335804
107891,32011.0,0.151213,103099.0,0.487017
108706,108596.0,0.512983,211695.0,1.000000


In [21]:
freq(data_prod.company_id, data_prod.data_permission_mask, cross = False, observed = True)

,,Count,Pct,Cuml Count,Cuml Pct
company_id,data_permission_mask,,,,
107671,1100000000000000000000000000000000000000000,27572.0,0.130244,27572.0,0.130244
108706,000000000100000000000000000000000000001000,108596.0,0.512983,136168.0,0.643227
107571,0000000001000000000000000000000000000000000,43516.0,0.205560,179684.0,0.848787
107891,1100000000000000000000000000000000000000000,32011.0,0.151213,211695.0,1.000000


In [19]:
freq(data_prod.company_id, data_prod.data_restriction_mask, cross = False, observed = True)

,,Count,Pct,Cuml Count,Cuml Pct
company_id,data_restriction_mask,,,,
107671,00001100010011000000000000000000000000000000000000,27572.0,0.130244,27572.0,0.130244
108706,0000010001001101,108596.0,0.512983,136168.0,0.643227
107571,00001100010011000000000000000000000000000,43516.0,0.205560,179684.0,0.848787
107891,00001100010011001000000010000000000000000000000000,32011.0,0.151213,211695.0,1.000000


In [4]:
## merged the data
keep = ["transaction_id", "date_added", "o_lexid"]
temp_prod = data_prod[keep]

keep =["acctno", "lexid"]
temp_retro = data[keep]
merged = temp_retro.merge(temp_prod, left_on = "acctno", right_on = "transaction_id")
print(merged.shape)

(211695, 5)


In [5]:
merged["lexid_change"] = np.where(merged.o_lexid == merged.lexid, 0, 1)
fmt = make_format(cuts = [-np.inf, 0, np.inf])
freq(merged.lexid_change, merged.o_lexid, merged.lexid, format = [None, fmt, fmt], cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
lexid_change o_lexid lexid                                            
0            1+      1+       208841.0  0.986518    208841.0  0.986518
1            <= 0    1+          428.0  0.002022    209269.0  0.988540
                     Missing    1629.0  0.007695    210898.0  0.996235
             1+      1+          505.0  0.002386    211403.0  0.998621
                     Missing     292.0  0.001379    211695.0  1.000000

In [80]:
merged[(merged.lexid_change == 1) & (merged.o_lexid<=0) & (merged.lexid.isnull())]

,acctno,lexid,transaction_id,date_added,o_lexid,lexid_change
51,436186501R97151,NaN,436186501R97151,2023-10-25 19:13:42,0,1
56,436186521R248785,NaN,436186521R248785,2023-10-28 23:55:41,0,1
220,436182181R842713,NaN,436182181R842713,2023-10-09 19:33:43,0,1
311,436182141R738027,NaN,436182141R738027,2023-10-06 22:35:41,0,1
422,436182251R653317,NaN,436182251R653317,2023-10-05 12:38:59,0,1
...,...,...,...,...,...,...
210665,30981,NaN,30981,2023-10-06 04:39:19,0,1
210966,31282,NaN,31282,2023-11-01 17:04:11,0,1
211211,31527,NaN,31527,2023-11-04 18:04:58,0,1
211515,31831,NaN,31831,2023-09-24 00:34:09,0,1


In [8]:
# temp = merged[merged.lexid_change == 0]
# print(temp.shape)
# data = data.merge(temp[["acctno", "transaction_id"]], on = "acctno")
data_prod = data_prod.merge(temp[["acctno", "transaction_id"]], on = "transaction_id")
print(data.shape)
print(data_prod.shape)

(208841, 537)
(208841, 407)


In [105]:
len([x for x in data.columns if "pl_" in x and x != "pl_fileaddrtype"])

310

In [18]:
col = [x for x in data.columns if "pl_" in x or "pi_" in x]
print(len(col))
not_in = ['PL_FileAddrTaxValYr', 'PL_FileAddrType', 'PL_DrgBkNewChType10Y', 'PL_DrgBkNewDispType10Y', 'PI_InpAddrTaxValYr', 'PI_InpAddrType']
not_in = [x.lower() for x in not_in]
data[[x for x in col if x not in not_in]] = data[[x for x in col if x not in not_in]].astype(float) 
data_prod[[x for x in col if x not in not_in]] = data_prod[[x for x in col if x not in not_in]].astype(float) 
col = col + ["acctno"]

340


In [19]:
from dsgtools.reporting import ImpactAnalysisReport
cr = ImpactAnalysisReport(data[col], data_prod[col], keys="acctno",
           exceptions= [-99_999, -99_998, -99_997],
           dataframe_names = ("retro", "prod"))
cr.to_xlsx("./_temp/riskview-comparison.xlsx", overwrite= True)

In [20]:
temp_prod = data_prod[data_prod.company_id == 107891]
print(temp_prod.shape)
from dsgtools.reporting import ImpactAnalysisReport
cr = ImpactAnalysisReport(data[col], temp_prod[col], keys="acctno",
           exceptions= [-99_999, -99_998, -99_997],
           dataframe_names = ("retro", "prod"))
cr.to_xlsx("./_temp/riskview-comparison_test.xlsx", overwrite= True)

(31772, 407)


In [41]:
samples = ['pl_srcoldmsncev', 'pl_inqcnt2y', "pl_inqcnt1m"]
temp_retro = data[samples + ['acctno', 'lexid']]
temp_prod = data_prod[samples + ['acctno', 'o_lexid', "company_id", "date_added"]]
merged = temp_retro.merge(temp_prod, on = "acctno")
print(merged.shape)

(208841, 11)


In [42]:
merged["flag"] = np.where(merged.lexid == merged.o_lexid, 0, 1)
freq(merged["flag"])

,Count,Pct,Cuml Count,Cuml Pct
flag,,,,
0,208841.0,1.0,208841.0,1.0


In [43]:
for x in samples:
    merged[x + "_diff"] = np.where((merged[x + "_x"] < 0) & (merged[x + "_y"] < 0), -999_999_999, np.where(
        (merged[x + "_x"] >= 0) & (merged[x + "_y"] < 0), -999_999_998, np.where(
            (merged[x + "_x"] < 0) & (merged[x + "_y"] >= 0), -999_999_997, merged[x+ "_x"] - merged[x + "_y"]
        )
    ))

In [44]:
temp = merged[merged["pl_inqcnt1m_diff"] >=2].head(25)
temp_2 = merged[merged["pl_inqcnt1m_diff"] >=2].tail(25)
pd.concat([temp, temp_2], ignore_index = True).to_csv("./_temp/example_for Kurtis_pl_inqcnt1m_diff_ge_2.csv", index = False)

In [45]:
pd.concat([temp, temp_2], ignore_index = True)

,pl_srcoldmsncev_x,pl_inqcnt2y_x,pl_inqcnt1m_x,acctno,lexid,pl_srcoldmsncev_y,pl_inqcnt2y_y,pl_inqcnt1m_y,o_lexid,company_id,date_added,flag,pl_srcoldmsncev_diff,pl_inqcnt2y_diff,pl_inqcnt1m_diff
0,50.0,25.0,2.0,436185801R398690,2.368264e+11,48.0,0.0,0.0,236826376871,108706,2023-10-20 17:59:43,0,2.0,25.0,2.0
1,301.0,23.0,2.0,436185821R193442,1.533377e+09,299.0,0.0,0.0,1533377188,108706,2023-10-16 22:05:53,0,2.0,23.0,2.0
2,233.0,15.0,2.0,436185821R291999,4.261070e+10,231.0,13.0,0.0,42610697954,107571,2023-10-18 19:24:43,0,2.0,2.0,2.0
3,235.0,25.0,3.0,4648493665R47174,4.381500e+10,233.0,0.0,0.0,43814997504,108706,2023-10-26 00:17:59,0,2.0,25.0,3.0
4,127.0,11.0,3.0,436182281R642105,1.904372e+11,125.0,0.0,0.0,190437189836,108706,2023-10-05 01:51:00,0,2.0,11.0,3.0
5,251.0,18.0,2.0,436182511R829096,1.238191e+10,249.0,0.0,0.0,12381907153,108706,2023-10-09 14:26:06,0,2.0,18.0,2.0
6,400.0,24.0,2.0,436182531R500413,2.004866e+08,398.0,0.0,0.0,200486572,108706,2023-10-02 16:49:46,0,2.0,24.0,2.0
7,205.0,5.0,5.0,436182541R976985,4.378787e+10,203.0,0.0,0.0,43787866340,108706,2023-10-12 04:51:20,0,2.0,5.0,5.0
8,456.0,7.0,2.0,436182161R607112,1.918570e+09,454.0,0.0,0.0,1918569591,108706,2023-10-04 15:42:26,0,2.0,7.0,2.0
9,58.0,13.0,4.0,436182551R612389,2.365405e+11,56.0,0.0,0.0,236540531613,108706,2023-10-04 17:15:24,0,2.0,13.0,4.0


In [35]:
fmt_2 = make_format(cuts = [-np.inf, -51, -21, -16, -11, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 10, 15, 20, 50, np.inf], exceptions = [-999_999_999, -999_999_998, -999_999_997])
freq('pl_inqcnt2y_diff', format = fmt_2, df = merged[merged.company_id == 108706 ])

,Count,Pct,Cuml Count,Cuml Pct
pl_inqcnt2y_diff,,,,
<= -51,0.0,0.000000,0.0,0.000000
-50--21,0.0,0.000000,0.0,0.000000
-20--16,0.0,0.000000,0.0,0.000000
-15--11,0.0,0.000000,0.0,0.000000
-10--6,0.0,0.000000,0.0,0.000000
-5,0.0,0.000000,0.0,0.000000
-4,0.0,0.000000,0.0,0.000000
-3,0.0,0.000000,0.0,0.000000
-2,0.0,0.000000,0.0,0.000000
